## 👉 START HERE: How to use this notebook

### Step 1: Create synthetic evaluation data

To measure your Agent's quality, you need a diverse, representative evaluation set.  This notebook turns your unstructured documents into a high-quality synthetic evaluation set so that you can start to evaluate and improve your Agent's quality before subject matter experts are available to label data.

This notebook does the following:
1. <TODO>

THIS DOES NOT WORK FROM LOCAL IDE YET.

**Important note:** Throughout this notebook, we indicate which cells you:
- ✅✏️ *should* customize - these cells contain config settings to change
- 🚫✏️ *typically will not* customize - these cells contain  code that is parameterized by your configuration.

*Cells that don't require customization still need to be run!*

### 🚫✏️ Install Python libraries

In [0]:
%pip install -qqqq -U -r requirements.txt
dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.24 which is incompatible.
langchain 0.1.20 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.2.2 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.24 which is incompatible.
langchain-community 0.0.38 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.2.2 which is incompatible.
langchain-text-splitters 0.0.2 requires langchain-core<0.3,>=0.1.28, but you have langchain-core 0.3.24 which is incompatible.
ydata-profiling 4.5.1 requires pandas!=1.4.0,<2.1,>1.1, but you have pandas 2.2.3 which is incompatible.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.10.3 which is incompatible.
Note: you may need to restart the

### 🚫✏️ Connect to Databricks

If running locally in an IDE using Databricks Connect, connect the Spark client & configure MLflow to use Databricks Managed MLflow.  If this running in a Databricks Notebook, these values are already set.

In [0]:
from mlflow.utils import databricks_utils as du
import os

if not du.is_in_databricks_notebook():
    from databricks.connect import DatabricksSession

    spark = DatabricksSession.builder.getOrCreate()
    os.environ["MLFLOW_TRACKING_URI"] = "databricks"

### 🚫✏️ Load the Agent's storage locations

This notebook writes to the evaluation set table that you specified in the [Agent setup](02_agent_setup.ipynb) notebook.

In [0]:
from cookbook.config.shared.agent_storage_location import AgentStorageConfig
from cookbook.databricks_utils import get_table_url
from cookbook.config import load_serializable_config_from_yaml_file

# Load the Agent's storage configuration
agent_storage_config: AgentStorageConfig = load_serializable_config_from_yaml_file('./configs/agent_storage_config.yaml')

# Check if the evaluation set already exists
try:
    eval_dataset = spark.table(agent_storage_config.evaluation_set_uc_table)
    if eval_dataset.count() > 0:
        print(f"Evaluation set {get_table_url(agent_storage_config.evaluation_set_uc_table)} already exists!  By default, this notebook will append to the evaluation dataset.  If you would like to overwrite the existing evaluation set, please delete the table before running this notebook.")
    else:
        print(f"Evaluation set {get_table_url(agent_storage_config.evaluation_set_uc_table)} exists, but is empty!  By default, this notebook will NOT change the schema of this table - if you experience schema related errors, drop this table before running this notebook so it can be recreated with the correct schema.")
except Exception:
    print(f"Evaluation set `{agent_storage_config.evaluation_set_uc_table}` does not exist.  This notebook will create a new Delta Table at this location.")

Evaluation set `casaman_ssa.demos.my_agent_autogen_eval_set` does not exist.  This notebook will create a new Delta Table at this location.


#### ✅✏️ Load the source documents for synthetic evaluation data generation

Most often, this will be the same as the document output table from the [data pipeline](01_data_pipeline.ipynb).

Here, we provide code to load the documents table that was created in the [data pipeline](01_data_pipeline.ipynb).

Alternatively, this can be a Spark DataFrame, Pandas DataFrame, or list of dictionaries with the following keys/columns:
- `doc_uri`: A URI pointing to the document.
- `content`: The content of the document.

In [0]:
from cookbook.config.data_pipeline import DataPipelineConfig
from cookbook.config import load_serializable_config_from_yaml_file

datapipeline_config: DataPipelineConfig= load_serializable_config_from_yaml_file('./configs/data_pipeline_config.yaml')

source_documents = spark.table(datapipeline_config.output.parsed_docs_table)

display(source_documents.toPandas())

content,parser_status,doc_uri,last_modified


#### ✅✏️ Run the synthetic evaluation data generation

Optionally, you can customize the guidelines to guide the synthetic data generation.  By default, guidelines are not applied - to apply the guidelines, uncomment `guidelines=guidelines` in the `generate_evals_df(...)` call.  See our [documentation](https://docs.databricks.com/en/generative-ai/agent-evaluation/synthesize-evaluation-set.html) for more details.

In [0]:
from databricks.agents.evals import generate_evals_df

# NOTE: The guidelines you provide are a free-form string. The markdown string below is the suggested formatting for the set of guidelines, however you are free
# to add your sections here. Note that this will be prompt-engineering an LLM that generates the synthetic data, so you may have to iterate on these guidelines before
# you get the results you desire.
guidelines = """
# Task Description
The Agent is a RAG chatbot that answers questions about using Spark on Databricks. The Agent has access to a corpus of Databricks documents, and its task is to answer the user's questions by retrieving the relevant docs from the corpus and synthesizing a helpful, accurate response. The corpus covers a lot of info, but the Agent is specifically designed to interact with Databricks users who have questions about Spark. So questions outside of this scope are considered irrelevant.

# User personas
- A developer who is new to the Databricks platform
- An experienced, highly technical Data Scientist or Data Engineer

# Example questions
- what API lets me parallelize operations over rows of a delta table?
- Which cluster settings will give me the best performance when using Spark?

# Additional Guidelines
- Questions should be succinct, and human-like
"""

synthesized_evals_df = generate_evals_df(
    docs=source_documents,
    # The number of evaluations to generate for each doc.
    num_evals=10,
    # A optional set of guidelines that help guide the synthetic generation. This is a free-form string that will be used to prompt the generation.
    # guidelines=guidelines
)

# Write the synthetic evaluation data to the evaluation set table
spark.createDataFrame(synthesized_evals_df).write.format("delta").mode("append").saveAsTable(agent_storage_config.evaluation_set_uc_table)

# Display the synthetic evaluation data
eval_set_df = spark.table(agent_storage_config.evaluation_set_uc_table)
display(eval_set_df.toPandas())

Generating evaluations:   0%|          | 0/10 evals generated [Elapsed: 00:00, Remaining: ?]

request_id request expected_retrieved_context expected_facts source_type source_id de1daac1a320379ce055bdc8b8342a2d7ca8d1ea08483081801f8219f41dc69d List(List(List(What percentage of consumers, according to a McKinsey study, are more likely to consider buying from a brand that personalizes the shopping and user experience?, user))) List(List(“In today’s experience-driven world, the most beloved brands are the ones that know their customers. Customers are loyal to brands that recognize their needs and preferences — and tailor user journeys and engagements accordingly.

A study from McKinsey shows 76% of consumers are more likely to consider buying from a brand that personalizes the shopping and user experience to the wants and needs of the customer. And as organizations pursue omnichannel excellence, these same high expectations of online experiences also extend to brick-and-mortar locations — revealing for many merchants that personalized engagement is fundamental to attracting customers and expanding share of wallet.

But achieving a 360-degree view of your customers to serve personalized experiences requires integrating various types of data — including demographics, behavioral and transactional — to develop robust profiles. This guide focuses on six actionable strategic pillars for businesses to leverage automation, real-time data, AI-driven analysis and well-tuned ML models to architect and deliver customized customer experiences at every touch point.”, /Volumes/casaman_ssa/demos/volume_databricks_documentation/databricks-pdf/Databricks-Customer-360-ebook-Final.pdf)) List(76% of consumers are more likely to consider buying from a brand that personalizes the shopping and user experience.) SYNTHETIC_FROM_DOC /Volumes/casaman_ssa/demos/volume_databricks_documentation/databricks-pdf/Databricks-Customer-360-ebook-Final.pdf 4b452a4426892dea5c35302c50dc70d62c0b2993f478af59a42b59d7c258bfa0 List(List(List(What are two key challenges mentioned for predictive maintenance in government agencies?, user))) List(List(##### Overview

**Integrating unstructured data**
Equipment data doesn’t just come in the form of IoT data. Agencies can gather rich unstructured signals like audio, visual (e.g., video inspections) and text (e.g., maintenance logs). Most legacy data architectures are unable to integrate structured and unstructured data sources.

**Operationalizing machine learning**
Most agencies lack the advanced analytics tools needed to build models that can predict potential equipment failures. Those that do typically have their data scientists working in a siloed set of tools, resulting in unnecessary data replication and inefficient workflows., /Volumes/casaman_ssa/demos/volume_databricks_documentation/databricks-pdf/big-book-of-data-and-ai-use-cases-for-the-public-sector.pdf)) List(Difficulty integrating structured and unstructured data sources due to legacy data architectures., Inefficient workflows caused by a lack of advanced analytics tools and siloed environments for data scientists.) SYNTHETIC_FROM_DOC /Volumes/casaman_ssa/demos/volume_databricks_documentation/databricks-pdf/big-book-of-data-and-ai-use-cases-for-the-public-sector.pdf 6d1c05783fb5945cc9b121919eabdc2194c9c64809821e3c30b7f758a4d12a40 List(List(List(What percentage of specialized Python libraries in the data set are associated with natural language processing (NLP), and what are some of the tasks enabled by NLP?, user))) List(List(```
Our most popular use case is natural language processing
(NLP), a rapidly growing field that enables businesses to
gain value from unstructured textual data. This opens the
door for users to accomplish tasks that were previously
too abstract for code, such as summarizing content or
extracting sentiment from customer reviews. In our data
set, 49% of libraries used are associated with NLP. LLMs
also fall within this bucket. Given the innovations launched
in recent months, we expect to see NLP take off even
more in coming years as it is a